In [20]:
import pandas as pd
x=pd.read_csv("C:/Users/hemap/Downloads/Paper2/Sentiment.csv")
df=pd.DataFrame(x)
df

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [21]:
df.shape

(13871, 21)

In [32]:
df=df[['text','sentiment']]

In [33]:
df.duplicated().sum()

2891

In [34]:
df=df.drop_duplicates()

## Q1. Print the total number of positive and negative sentiments

In [35]:
df.drop(df[df['sentiment']=='Neutral'].index,inplace=True)

In [36]:
y=df['sentiment']

In [37]:
df.sentiment.value_counts()


Negative    6129
Positive    1709
Name: sentiment, dtype: int64

## Q2. Build a sequential LSTM model to predict positive and negative sentiments.


In [39]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hemap\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
word_corpus = []
for text in df['text']:
    words = [word.lower() for word in word_tokenize(text)] 
    word_corpus.append(words)
numberof_words = len(word_corpus)
print(numberof_words)

7838


In [47]:
from nltk.tokenize import word_tokenize
token=word_tokenize(text)
print(token)

['RT', '@', 'georgehenryw', ':', 'Who', 'thought', 'Huckabee', 'exceeded', 'their', 'expectations', '#', 'gopdebate', '#', 'imwithhuck', '#', 'gop', '#', 'ccot', '#', 'teaparty', '#', 'tcot', '@', 'laura4fairtax', 'http…']


In [50]:
from nltk.probability import FreqDist
fdist=FreqDist(token)

In [51]:
import textblob
from textblob import TextBlob
TextBlob(text).ngrams(2)

[WordList(['RT', 'georgehenryw']),
 WordList(['georgehenryw', 'Who']),
 WordList(['Who', 'thought']),
 WordList(['thought', 'Huckabee']),
 WordList(['Huckabee', 'exceeded']),
 WordList(['exceeded', 'their']),
 WordList(['their', 'expectations']),
 WordList(['expectations', 'gopdebate']),
 WordList(['gopdebate', 'imwithhuck']),
 WordList(['imwithhuck', 'gop']),
 WordList(['gop', 'ccot']),
 WordList(['ccot', 'teaparty']),
 WordList(['teaparty', 'tcot']),
 WordList(['tcot', 'laura4fairtax']),
 WordList(['laura4fairtax', 'http…'])]

In [ ]:
from nltk.stem import PorterStemmer
pst=PorterStemmer
from nltk.stem import LancasterStemmer
lst=LancasterStemmer()

In [53]:
from nltk import tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hemap\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [54]:
a=set(stopwords.words('english'))
t=word_tokenize(text.lower())
print(t)

['rt', '@', 'georgehenryw', ':', 'who', 'thought', 'huckabee', 'exceeded', 'their', 'expectations', '#', 'gopdebate', '#', 'imwithhuck', '#', 'gop', '#', 'ccot', '#', 'teaparty', '#', 'tcot', '@', 'laura4fairtax', 'http…']


In [55]:
stopwords=[x for x in text if x not in a]
print(stopwords)

['R', 'T', ' ', '@', 'g', 'e', 'r', 'g', 'e', 'h', 'e', 'n', 'r', 'w', ':', ' ', 'W', 'h', ' ', 'h', 'u', 'g', 'h', ' ', 'H', 'u', 'c', 'k', 'b', 'e', 'e', ' ', 'e', 'x', 'c', 'e', 'e', 'e', ' ', 'h', 'e', 'r', ' ', 'e', 'x', 'p', 'e', 'c', 'n', ' ', ' ', '\n', '\n', '#', 'g', 'p', 'e', 'b', 'e', ' ', '#', 'w', 'h', 'h', 'u', 'c', 'k', ' ', '#', 'g', 'p', ' ', '#', 'c', 'c', ' ', '#', 'e', 'p', 'r', ' ', '#', 'c', '\n', '@', 'l', 'u', 'r', '4', 'f', 'r', 'x', ' ', 'h', 'p', '…']


In [56]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
all_words= ' '.join([t for t in stopwords])

In [60]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [63]:
pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=847983a6cba8b07e6909a631f2f41563c26696f4ad4881f624778d2e1e9b8b1e
  Stored in directory: c:\users\hemap\appdata\local\pip\cache\wheels\b6\0d\90\0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
Successfully built termcolor
Note: you may need to restart the kernel to use updated packages.


In [64]:
import re
import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
df['text'] = df.text.apply(lemmatize_text)
df

In [ ]:
s = 0.0
for i in df['text']:
    word_list = i.split()
    s = s + len(word_list)
print("Average length of each text : ",s/df.shape[0])
pos = 0
for i in range(df.shape[0]):
    if df.iloc[i]['sentiment'] == 'positive':
        pos = pos + 1
neg = df.shape[0]-pos


## Q3 Checkthe Sentiment

In [ ]:

from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=5000)

sentence = ['He is a great leader','He is a terrible leader.' ]
# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)
# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)
# Get labels based on probability 1 if p>= 0.5 else 0
prediction = model.predict(padded)
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
for i in range(len(sentence)):
    print(sentence[i])
    if pred_labels[i] == 1:
        s = 'Positive'
    else:
        s = 'Negative'
    print("Predicted sentiment : ",s)